In [1]:
import xarray as xr 
import xcast as xc 
import matplotlib.pyplot 
import pandas as pd 
import cartopy.crs as ccrs 
import matplotlib.pyplot as plt 
import cartopy.feature as cfeature

# Parameters

In [45]:
ND=5
hidden_layer_size=5
activation = 'relu'
preprocessing = 'minmax'
target = 'WK23' # WK23 or WK34

# Load Data

In [46]:
# open data files
x23 = xr.open_dataset('ECMWF_{}.nc'.format(target)).tp
bd23 = xr.open_dataset('ENACTS-BD-{}.nc'.format(target)).rfe_merged
imd23 = xr.open_dataset('IMD-DAILY-{}.nc'.format(target)).rf

# regrid predictors onto predictands 
bd_x23 = xc.regrid(x23, bd23.coords['X'].values, bd23.coords['Y'].values)
imd_x23 = xc.regrid(x23, imd23.coords['lon'].values, imd23.coords['lat'].values)

# one-hot encode predictands for skill evaluation 
ohc = xc.RankedTerciles()
ohc.fit(imd23)
imd_ohc = ohc.transform(imd23)

ohc = xc.RankedTerciles()
ohc.fit(bd23)
bd_ohc = ohc.transform(bd23)

In [49]:
bd_x23.sel(S=slice(pd.Timestamp(2000,12, 31), None))

<xarray.DataArray (M: 10, S: 988, Y: 134, X: 108)>
array([[[[ 430.83650919,  449.80420327,  468.76900329, ...,
          2733.66459925, 2730.72558883, 2703.22410569],
         [ 442.20816503,  461.70822724,  481.20531415, ...,
          2741.29209109, 2735.43330576, 2705.78685465],
         [ 453.58025468,  473.61270531,  493.64209942, ...,
          2748.9198739 , 2740.14120228, 2708.34970137],
         ...,
         [1942.85051296, 1988.81119865, 2034.76487173, ...,
          2188.46585614, 2183.59723569, 2188.23572176],
         [1971.46541353, 2017.22673354, 2062.98107134, ...,
          2134.64056357, 2129.16265337, 2134.66444973],
         [1955.5603771 , 2000.48169771, 2045.3961643 , ...,
          2081.90329833, 2076.3670914 , 2082.62615767]],

        [[ 676.26101772,  692.50966565,  708.75583438, ...,
          4623.46272307, 4590.85381642, 4524.11512954],
         [ 662.28146072,  678.02664811,  693.76943311, ...,
          4625.38781881, 4590.56230866, 4522.44757789],
         [ 648.30137044,  663.54307807,  678.78246015, ...,
          4627.31298798, 4590.27078978, 4520.77996263],
...
         [ 224.25692011,  225.76866481,  227.28017886, ...,
           208.37656021,  208.72437131,  216.03234881],
         [ 231.1876174 ,  232.77823157,  234.36860305, ...,
           166.39685785,  166.45654637,  174.26101519],
         [ 231.21061306,  232.82114134,  234.43142388, ...,
           149.78322186,  149.90398892,  158.01690936]],

        [[ 670.8413845 ,  674.21852203,  677.59514429, ...,
          1051.32032667, 1051.01911727, 1046.41622133],
         [ 673.62711713,  677.11748033,  680.60731098, ...,
          1007.80510887, 1006.92320629, 1002.92947452],
         [ 676.41295603,  680.01654922,  683.61959258, ...,
           964.28823106,  962.82561316,  959.44106879],
         ...,
         [  47.2373741 ,   47.23328531,   47.22919715, ...,
            47.40886573,   45.86036107,   45.24315629],
         [  47.7921524 ,   47.78766427,   47.78317683, ...,
            43.86951634,   42.36107837,   41.68777947],
         [  47.65088575,   47.65841641,   47.66594592, ...,
            45.60989076,   44.27103537,   43.64352416]]]])
Coordinates:
  * Y        (Y) float32 20.38 20.42 20.48 20.52 ... 26.88 26.92 26.98 27.02
  * X        (X) float32 87.68 87.73 87.78 87.83 ... 92.88 92.93 92.97 93.03
  * M        (M) float32 1.0 2.0 3.0 4.0 5.0 6.0 7.0 8.0 9.0 10.0
  * S        (S) datetime64[ns] 2001-05-04 2001-05-07 ... 2019-10-26 2019-10-29
Attributes:
    pointwidth:          0
    gribPDSpattern:      04XXXX003D0134
    long_name:           Accumulated Total Precipitation
    units:               kg m-2
    standard_name:       precipitation_amount
    file_missing_value:  9.999e+20
    history:             Averaged over L[7.0 days, 21.0 days] minimum 0.0% da...
    generated_by:        \n  XCAST regridded

# Leave-One-Year-Out Cross Validation - WK23

In [36]:
imd_wk23_hcsts = []
bd_wk23_hcsts = []
start, i = pd.Timestamp.now(), 1
for year in range(2000, 2020):
    if year == 2000:
        bd_xtrain23 = bd_x23.sel(S=slice(pd.Timestamp(2000, 12, 31), None))
        bd_xtest23 = bd_x23.sel(S=slice(None, pd.Timestamp(2000, 12, 31)) )
        
        imd_xtrain23 = imd_x23.sel(S=slice(pd.Timestamp(2000, 12, 31), None))
        imd_xtest23 = imd_x23.sel(S=slice(None, pd.Timestamp(2000, 12, 31)) )
        
        imdtrain23 = imd23.sel(time=slice(pd.Timestamp(2000, 12, 31), None))
        imdtest23 = imd23.sel(time=slice(None,  pd.Timestamp(2000, 12, 31)))
        bdtrain23 = bd23.sel(T=slice( pd.Timestamp(2000, 12, 31), None))
        bdtest23 = bd23.sel(T=slice( None, pd.Timestamp(2000, 12, 31)))
    elif year == 2019: 
        bd_xtrain23 = bd_x23.sel(S=slice( None, pd.Timestamp(2018, 12, 31)))
        bd_xtest23 = bd_x23.sel(S=slice(pd.Timestamp(2018, 12, 31), None) )
        
        imd_xtrain23 = imd_x23.sel(S=slice( None, pd.Timestamp(2018, 12, 31)))
        imd_xtest23 = imd_x23.sel(S=slice(pd.Timestamp(2018, 12, 31), None) )
        
        imdtrain23 = imd23.sel(time=slice(None, pd.Timestamp(2018, 12, 31)))
        imdtest23 = imd23.sel(time=slice(pd.Timestamp(2018, 12, 31), None))
        bdtrain23 = bd23.sel(T=slice(None, pd.Timestamp(2018, 12, 31)))
        bdtest23 = bd23.sel(T=slice(pd.Timestamp(2018, 12, 31), None)) 
    else: 
        bd_xtrain23_low = bd_x23.sel(S=slice(None, pd.Timestamp(year-1, 12, 31)))
        bd_xtrain23_high = bd_x23.sel(S=slice( pd.Timestamp(year, 12, 31), None) )
        bd_xtrain23 = xr.concat([bd_xtrain23_low, bd_xtrain23_high], 'S')
        bd_xtest23 = bd_x23.sel(S=slice(pd.Timestamp(year-1, 12, 31), pd.Timestamp(year, 12, 31) ) )
        
        imd_xtrain23_low = imd_x23.sel(S=slice(None, pd.Timestamp(year-1, 12, 31)))
        imd_xtrain23_high = imd_x23.sel(S=slice( pd.Timestamp(year, 12, 31), None) )
        imd_xtrain23 = xr.concat([imd_xtrain23_low, imd_xtrain23_high], 'S')
        imd_xtest23 = imd_x23.sel(S=slice(pd.Timestamp(year-1, 12, 31), pd.Timestamp(year, 12, 31) ) )
        
        imdtrain23_low = imd23.sel(time=slice(None, pd.Timestamp(year-1, 12, 31)))
        imdtrain23_high = imd23.sel(time=slice(pd.Timestamp(year, 12, 31), None))
        imdtrain23 = xr.concat([imdtrain23_low, imdtrain23_high], 'time')
        imdtest23 = imd23.sel(time=slice(pd.Timestamp(year-1, 12, 31), pd.Timestamp(year, 12, 31) ) )
        
        bdtrain23_low = bd23.sel(T=slice(None, pd.Timestamp(year-1, 12, 31)))
        bdtrain23_high = bd23.sel(T=slice(pd.Timestamp(year, 12, 31), None))
        bdtrain23 = xr.concat([bdtrain23_low, bdtrain23_high], 'T')
        bdtest23 = bd23.sel(T=slice(pd.Timestamp(year-1, 12, 31), pd.Timestamp(year, 12, 31) ) )

    imdt_ohc = xc.RankedTerciles()
    imdt_ohc.fit(imdtrain23)
    imd_ohc_y_train = imdt_ohc.transform(imdtrain23)
    

    imdpoelm = xc.cPOELM(ND=ND, hidden_layer_size=hidden_layer_size, activation=activation, preprocessing=preprocessing)
    imdpoelm.fit(imd_xtrain23, imd_ohc_y_train)
    imdpoelm_preds = imdpoelm.predict(imd_xtest23)
    imd_wk23_hcsts.append(imdpoelm_preds)
    
    bdt_ohc = xc.RankedTerciles()
    bdt_ohc.fit(bdtrain23)
    bd_ohc_y_train = bdt_ohc.transform(bdtrain23)
    
    bdpoelm = xc.cPOELM(ND=ND, hidden_layer_size=hidden_layer_size, activation=activation, preprocessing=preprocessing)
    bdpoelm.fit(bd_xtrain23, bd_ohc_y_train)
    bdpoelm_preds = bdpoelm.predict(bd_xtest23)
    bd_wk23_hcsts.append(bdpoelm_preds)
    
    print('Fiting Window {}/20 - {}'.format(i, pd.Timestamp.now() - start), end='\r')
    i+= 1
bd_wk23 = xr.concat(bd_wk23_hcsts, 'S')
imd_wk23 = xr.concat(imd_wk23_hcsts, 'S')


KeyError: Timestamp('2000-12-31 00:00:00')

# Calculate & Show Skill 

In [ ]:
start = pd.Timestamp.now()
bd = bd_wk23.mean('ND')
bdgroc = xc.GeneralizedROC(bd, bd_ohc)
bdpoelm_rps = xc.RankProbabilityScore(bd, bd_ohc)
bdclimatological_odds = xr.ones_like(bd) * 0.33 
bdclimo_rps = xc.RankProbabilityScore(bdclimatological_odds, bd_ohc)
bdrpss = 1 - ( bdpoelm_rps / bdclimo_rps)
print(pd.Timestamp.now() - start)

In [ ]:
cmap = plt.get_cmap('autumn_r')
cmap.set_under('lightgray')

fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(10, 5), subplot_kw={'projection': ccrs.PlateCarree()})
bdgroc.plot(ax=ax[0], vmin=0.5, vmax=1, cmap=cmap)
states_provinces = cfeature.NaturalEarthFeature(category='cultural', name='admin_0_countries', scale='50m', facecolor='none')
ax[0].add_feature(states_provinces)
ax[0].set_title('Generalized ROC')

bdrpss.plot(ax=ax[1], vmin=0, vmax=0.2, cmap=cmap)
ax[1].add_feature(states_provinces)
ax[1].set_title('Rank Probability Skill Score')
plt.show()

In [ ]:
start = pd.Timestamp.now()
imd = imd_wk23.mean('ND')
imdgroc = xc.GeneralizedROC(imd, imd_ohc)
imdpoelm_rps = xc.RankProbabilityScore(imd, imd_ohc)
imdclimatological_odds = xr.ones_like(imd) * 0.33 
imdclimo_rps = xc.RankProbabilityScore(imdclimatological_odds, imd_ohc)
imdrpss = 1 - ( imdpoelm_rps / imdclimo_rps)
print(pd.Timestamp.now() - start)

In [ ]:
cmap = plt.get_cmap('autumn_r')
cmap.set_under('lightgray')

fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(10, 5), subplot_kw={'projection': ccrs.PlateCarree()})
imdgroc.plot(ax=ax[0], vmin=0.5, vmax=1, cmap=cmap)
states_provinces = cfeature.NaturalEarthFeature(category='cultural', name='admin_0_countries', scale='50m', facecolor='none')
ax[0].add_feature(states_provinces)
ax[0].set_title('Generalized ROC')

imdrpss.plot(ax=ax[1], vmin=0, vmax=0.2, cmap=cmap)
ax[1].add_feature(states_provinces)
ax[1].set_title('Rank Probability Skill Score')
plt.show()